In [1]:
# Check PyTorch installation
import torch, torchvision
print(torch.__version__)
print(torch.cuda.is_available())

1.10.1
True


In [135]:
%%writefile configs/resnet/resnet18_alzheimer_axial_view.py
_base_ = [
    '../_base_/models/resnet18.py', 
    # '../_base_/schedules/cifar10_bs128.py', 
    '../_base_/schedules/imagenet_bs1024_adamw_conformer.py',
    # '../_base_/schedules/imagenet_bs256.py',
    '../_base_/default_runtime.py'
]

pretrained='https://download.openmmlab.com/mmclassification/v0/resnet/resnet18_b16x8_cifar10_20210528-bd6371c8.pth'

model = dict(
    type='ImageClassifier',
    backbone=dict(
        type='ResNet', 
        style='pytorch',
        init_cfg=dict(
            type='Pretrained',
            checkpoint=pretrained,
            prefix='backbone'
        ),
    ),
    head=dict(
        num_classes=3,
        topk=(1, 3),
    ))

dataset_type = 'CustomDataset'
# dataset_type = 'ImageNet'
data_preprocessor = dict(
     # mean=[124.508, 116.050, 106.438],
     # std=[58.577, 57.310, 57.437],
     mean=[124.508, 116.050, 106.438],
     std=[58.577, 57.310, 57.437],
     to_rgb=False)

# data_preprocessor = dict(
#     # Input image data channels in 'RGB' order
#     mean=[123.675, 116.28, 103.53],    # Input image normalized channel mean in RGB order
#     std=[58.395, 57.12, 57.375],       # Input image normalized channel std in RGB order
#     to_rgb=True,                       # Whether to flip the channel from BGR to RGB or RGB to BGR
# )
train_pipeline = [
    dict(type='LoadImageFromFile'),     # read image
    # dict(type='ResizeEdge', scale=256),  # Scale the short side to 256
    # dict(type='RandomResizedCrop', scale=224), # 224x224
    dict(type='RandomFlip', prob=0.5, direction='horizontal'),   # random horizontal flip
    dict(type='PackInputs'),
]
test_pipeline = [
    dict(type='LoadImageFromFile'),     # read image
    # dict(type='ResizeEdge', scale=256),  # Scale the short side to 256
    # dict(type='CenterCrop', crop_size=224),
    dict(type='PackInputs'),    
]

train_dataloader = dict(
    batch_size=32,
    num_workers=5,
    dataset=dict(
        type=dataset_type,
        data_root='data',
        data_prefix='YoriDataset_vgg/train',
        ann_file='train_ann.txt',
        classes='data/classes.txt',
        with_label=True,
        pipeline=train_pipeline
    ),
    sampler=dict(type='DefaultSampler', shuffle=True),
    persistent_workers=True
)

val_dataloader = dict(
    batch_size=32,
    num_workers=5,
    dataset=dict(
        type=dataset_type,
        data_root='data',
        data_prefix='YoriDataset_vgg/validation',
        classes='data/classes.txt',
        ann_file='val_ann.txt',
        with_label=True,
        pipeline=test_pipeline
    ),
    sampler=dict(type='DefaultSampler', shuffle=False),
    persistent_workers=True
)

test_dataloader = dict(
    batch_size=32,
    num_workers=5,
    dataset=dict(
        type=dataset_type,
        data_root='data',
        data_prefix='YoriDataset_vgg/test',
        classes='data/classes.txt',
        ann_file='test_ann.txt',
        with_label=True,
        pipeline=test_pipeline
    ),    
    sampler=dict(type='DefaultSampler', shuffle=False),
    persistent_workers=True
)

val_evaluator = dict(type='Accuracy', topk=(1, 3))
test_evaluator = val_evaluator 

# optim_wrapper = dict(
#     # Use SGD optimizer to optimize parameters.
#     optimizer=dict(type='SGD', lr=0.1, momentum=0.9, weight_decay=0.0001))

# # The tuning strategy of the learning rate.
# # The 'MultiStepLR' means to use multiple steps policy to schedule the learning rate (LR).
# param_scheduler = dict(
#     type='MultiStepLR', by_epoch=True, milestones=[30, 60, 90], gamma=0.1)

optim_wrapper = dict(
    optimizer=dict(
        type='AdamW',
        # for batch in each gpu is 128, 8 gpu
        # lr = 5e-4 * 128 * 8 / 512 = 0.001
        lr=5e-4 * 128 * 8 / 512,
        weight_decay=0.05,
        eps=1e-8,
        betas=(0.9, 0.999)),
    paramwise_cfg=dict(
        norm_decay_mult=0.0,
        bias_decay_mult=0.0,
        custom_keys={
            '.cls_token': dict(decay_mult=0.0),
        }),
)

# learning policy
param_scheduler = [
    dict(
        type='LinearLR',
        start_factor=1e-3,
        by_epoch=True,
        begin=0,
        end=5,
        convert_to_iter_based=True),
    dict(
        type='CosineAnnealingLR',
        T_max=295,
        eta_min=1e-5,
        by_epoch=True,
        begin=5,
        end=300)
]

train_cfg = dict(by_epoch=True, max_epochs=3, val_interval=1)
val_cfg = dict()
test_cfg = dict()

auto_scale_lr = dict(base_batch_size=256)

default_scope = 'mmpretrain'

# configure default hooks
default_hooks = dict(
    # record the time of every iteration.
    timer=dict(type='IterTimerHook'),

    # print log every 100 iterations.
    logger=dict(type='LoggerHook', interval=100),

    # enable the parameter scheduler.
    param_scheduler=dict(type='ParamSchedulerHook'),

    # save checkpoint per epoch.
    checkpoint=dict(type='CheckpointHook', interval=1),

    # set sampler seed in a distributed environment.
    sampler_seed=dict(type='DistSamplerSeedHook'),

    # validation results visualization, set True to enable it.
    visualization=dict(type='VisualizationHook', enable=False),
)

env_cfg = dict(
    # whether to enable cudnn benchmark
    cudnn_benchmark=False,

    # set multi-process parameters
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0),

    # set distributed parameters
    dist_cfg=dict(backend='nccl'),
)

# set visualizer
vis_backends = [dict(type='LocalVisBackend')]  # use local HDD backend
visualizer = dict(type='UniversalVisualizer', vis_backends=vis_backends, name='visualizer')

# set log level
log_level = 'INFO'

# load from which checkpoint
load_from = None

# whether to resume training from the loaded checkpoint
resume = False

Overwriting configs/resnet/resnet18_alzheimer_axial_view.py


In [134]:
!python ./tools/train.py \
    configs/resnet/resnet18_alzheimer_axial_view.py \
    --work-dir work_dirs/alzheimer/axial/resnet18 \
    # --auto-scale-lr

configs/resnet/resnet18_alzheimer_axial_view.py
11/18 16:55:29 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.18 (default, Sep 11 2023, 13:40:15) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 360360474
    GPU 0: NVIDIA GeForce RTX 3090
    CUDA_HOME: /usr/local/cuda-11.4
    NVCC: Cuda compilation tools, release 11.4, V11.4.100
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
    PyTorch: 1.10.1
    PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.2.3 (Git Hash 7336ca9f055cf1bfa13efb658fe15dc9b41f0740)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.3
  - NVCC architecture flags: -gencode;ar

In [118]:
import mmcv
from mmpretrain.apis import inference_model, init_model

config_file = 'configs/resnet/resnet18_alzheimer_axial_view.py'
checkpoint_file = 'work_dirs/alzheimer/axial/resnet18/epoch_100.pth'

# Specify the device, if you cannot use GPU, you can also use CPU 
# by specifying `device='cpu'`.
device = 'cuda:0'
# device = 'cpu'

# Build the model according to the config file and load the checkpoint.
model_axial = init_model(config_file, checkpoint_file, device=device)
model_axial.backbone.layer4

Loads checkpoint by local backend from path: work_dirs/alzheimer/axial/resnet18/epoch_100.pth


ResLayer(
  (0): BasicBlock(
    (conv1): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (drop_path): Identity()
  )
  (1): BasicBlock(
    (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(512, eps=1e-05,

In [120]:
image_url='data/YoriDataset_vgg/train/MCI/100x-103.png'
image = np.array(Image.open(image_url))
image = (transforms.ToTensor()(image))

image.shape

torch.Size([1, 192, 160])

In [121]:
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image
from torchvision.models import resnet18
from torchvision import transforms
from PIL import Image
import numpy as np

# model = resnet50(pretrained=True)
model = model_axial

target_layers = [model.backbone.layer4[-1]]
image_url='data/YoriDataset_vgg/train/MCI/100x-103.png'
# image_url='demo/cat-dog.png'
image = np.array(Image.open(image_url))
# image = (transforms.ToTensor()(image))
rgb_img = np.float32(image) / 255
print(rgb_img.shape)
input_tensor = preprocess_image(
                                rgb_img,
                                mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225]
                                )
# Construct the CAM object once, and then re-use it on many images:
with GradCAM(model=model,
             target_layers=target_layers,
             use_cuda=torch.cuda.is_available()) as cam:
    
    grayscale_cam = cam(input_tensor=input_tensor, targets=target_layers)[0,:]
    visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

(192, 160)


RuntimeError: output with shape [1, 192, 160] doesn't match the broadcast shape [3, 192, 160]

In [125]:
!python tools/visualization/vis_cam.py \
    data/YoriDataset_vgg/train/MCI/100x-103.png \
    configs/resnet/resnet18_alzheimer_axial_view.py \
    work_dirs/alzheimer/axial/resnet18/epoch_100.pth \
    # --target-layers backbone.layer4.1 \
    --target-layers backbone.layer4.1.bn2 \
    --method GradCAM
    # GradCAM++, XGradCAM, EigenCAM, EigenGradCAM, LayerCAM

Loads checkpoint by local backend from path: work_dirs/alzheimer/axial/resnet18/epoch_100.pth
20
Automatically choose the last norm layer "backbone.layer4.1.bn2" as the target layer.


In [138]:
import os
import subprocess
train_AD = 'data/YoriDataset_vgg/train/AD'
train_MCI = 'data/YoriDataset_vgg/train/MCI'
train_NC = 'data/YoriDataset_vgg/train/NC'

val_AD = 'data/YoriDataset_vgg/validation/AD'
val_MCI = 'data/YoriDataset_vgg/validation/MCI'
val_NC = 'data/YoriDataset_vgg/validation/NC'

test_AD = 'data/YoriDataset_vgg/test/AD'
test_MCI = 'data/YoriDataset_vgg/test/MCI'
test_NC = 'data/YoriDataset_vgg/test/NC'


trainheat_AD = 'data/resnet_heat/train/AD'
trainheat_MCI = 'data/resnet_heat/train/MCI'
trainheat_NC = 'data/resnet_heat/train/NC'

valheat_AD = 'data/resnet_heat/validation/AD'
valheat_MCI = 'data/resnet_heat/validation/MCI'
valheat_NC = 'data/resnet_heat/validation/NC'

testheat_AD = 'data/vgg_heat/test/AD'
testheat_MCI = 'data/vgg_heat/test/MCI'
testheat_NC = 'data/vgg_heat/test/NC'

count = 0
for i in os.listdir(train_NC):
    count+=1
    temp = os.path.join(train_NC,i)
    out = trainheat_NC + '/heat_'+i
    print(out)  
    subprocess.run(["python", "tools/visualization/vis_cam.py", 
                temp,
                 "configs/resnet/resnet18_alzheimer_axial_view.py",
                 "work_dirs/alzheimer/axial/resnet18/epoch_100.pth",
                 "--target-layers", "backbone.layer4.1.bn2",
                 "--method", "GradCAM++",
                 "--save-path", out,
                 # "--eigen-smooth", "--aug-smooth"
                 ], text=True)
    print(count)
    # break

data/resnet_heat/train/NC/heat_63x-122.png
Loads checkpoint by local backend from path: work_dirs/alzheimer/axial/resnet18/epoch_100.pth
1
data/resnet_heat/train/NC/heat_26x-110.png
Loads checkpoint by local backend from path: work_dirs/alzheimer/axial/resnet18/epoch_100.pth
2
data/resnet_heat/train/NC/heat_72x-135.png
Loads checkpoint by local backend from path: work_dirs/alzheimer/axial/resnet18/epoch_100.pth
3
data/resnet_heat/train/NC/heat_14x-112.png
Loads checkpoint by local backend from path: work_dirs/alzheimer/axial/resnet18/epoch_100.pth
4
data/resnet_heat/train/NC/heat_7x-142.png
Loads checkpoint by local backend from path: work_dirs/alzheimer/axial/resnet18/epoch_100.pth
5
data/resnet_heat/train/NC/heat_37x-156.png
Loads checkpoint by local backend from path: work_dirs/alzheimer/axial/resnet18/epoch_100.pth
6
data/resnet_heat/train/NC/heat_25x-100.png
Loads checkpoint by local backend from path: work_dirs/alzheimer/axial/resnet18/epoch_100.pth
7
data/resnet_heat/train/NC/he